# Environmental Mapping Playground

In [1]:
import os
import geemap
import numpy as np
import geopandas as gpd

In [2]:
nh_center = [43.78699687528447, -71.51658744025995]
nh_zoom = 8

In [13]:
# Helper function to convert square feet to acres
def sq_ft_to_acres(sq_ft):
    """
    Returns area in acres when square feet are input.
    :param sq_ft: numeric type, area in square feet
    :return: float, area in acres
    """
    conv_factor = 43560.0
    return sq_ft / conv_factor

## Test Import of Political Boundaries GeoJSON

In [3]:
pb_poly_url = 'https://opendata.arcgis.com/datasets/4edf75ab263b4d92996f92fb9cf435fa_8.geojson'

In [4]:
pb_poly = gpd.read_file(pb_poly_url, driver='geojson')
pb_poly.head()

,pbpOBJECTID,pbpFIPS,pbpCOUSUB,pbpNAME,pbpRPA,pbpCOUNTY,pbpSHAPE_Length,pbpSHAPE_Area,PB_TOWN_Census_2010_StatsOBJECTID,PB_TOWN_Census_2010_StatsFIPS,...,PB_TOWN_Census_2010_StatsFIPS_1,PB_TOWN_Census_2010_StatsALTTOWNNAME,PB_TOWN_Census_2010_StatsNBRFIPS,PB_TOWN_Census_2010_StatsCOUNTYNAME_1,PB_TOWN_Census_2010_StatsRPASHORT,PB_TOWN_Census_2010_StatsPOP100,PB_TOWN_Census_2010_StatsHU100,PB_TOWN_Census_2010_StatsPOPMALE,PB_TOWN_Census_2010_StatsPOPFEMALE,geometry
0,1,7215,84420,Whitefield,1,7,159779.021915,9.684227e+08,41,7215,...,7215,Whitefield,7215,Coos,North Country Council,2306,1339,1126,1180,"POLYGON ((-71.62844 44.41026, -71.62844 44.409..."
1,2,7180,68980,Shelburne,1,7,147042.874299,1.359590e+09,4,7180,...,7180,Shelburne,7180,Coos,North Country Council,372,217,192,180,"POLYGON ((-71.16158 44.43815, -71.16158 44.436..."
2,3,7075,18420,Dixville,1,7,166006.939835,1.370192e+09,62,7075,...,7075,Dixville,7075,Coos,North Country Council,12,33,6,6,"POLYGON ((-71.29019 44.97358, -71.29117 44.969..."
3,4,7070,18340,Dixs Grant,1,7,105578.847104,5.594624e+08,144,7070,...,7070,Dixs Grant,7070,Coos,North Country Council,1,15,0,1,"POLYGON ((-71.22457 44.96439, -71.22457 44.964..."
4,5,7045,13780,Colebrook,1,7,165905.674698,1.137201e+09,135,7045,...,7045,Colebrook,7045,Coos,North Country Council,2301,1429,1133,1168,"POLYGON ((-71.51525 44.94274, -71.51525 44.942..."


In [5]:
pb_poly.shape

(259, 29)

In [7]:
pb_poly.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [9]:
pb_poly = pb_poly.to_crs('epsg:3437')

<Derived Projected CRS: EPSG:3437>
Name: NAD83 / New Hampshire (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - New Hampshire - counties of Belknap; Carroll; Cheshire; Coos; Grafton; Hillsborough; Merrimack; Rockingham; Strafford; Sullivan.
- bounds: (-72.56, 42.69, -70.63, 45.31)
Coordinate Operation:
- name: SPCS83 New Hampshire zone (US Survey feet)
- method: Transverse Mercator
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [6]:
pb_poly.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 29 columns):
 #   Column                                 Non-Null Count  Dtype   
---  ------                                 --------------  -----   
 0   pbpOBJECTID                            259 non-null    int64   
 1   pbpFIPS                                259 non-null    int64   
 2   pbpCOUSUB                              259 non-null    int64   
 3   pbpNAME                                259 non-null    object  
 4   pbpRPA                                 259 non-null    int64   
 5   pbpCOUNTY                              259 non-null    int64   
 6   pbpSHAPE_Length                        259 non-null    float64 
 7   pbpSHAPE_Area                          259 non-null    float64 
 8   PB_TOWN_Census_2010_StatsOBJECTID      259 non-null    int64   
 9   PB_TOWN_Census_2010_StatsFIPS          259 non-null    int64   
 10  PB_TOWN_Census_2010_StatsNAME          259 non-null   

In [11]:
cols_to_keep = ['pbpFIPS',
                'pbpCOUSUB',
                'pbpNAME',
                'pbpRPA',
                'pbpCOUNTY',
                'geometry']

pb_poly = pb_poly.loc[:, cols_to_keep].copy()

In [27]:
# Clean column names
pb_poly.columns = ['FIPS',
                  'COUSUB',
                  'TOWN_NAME',
                  'RPA',
                  'COUNTY',
                  'geometry']

In [29]:
# Add acreage column
pb_poly['TOWN_ACRES'] = pb_poly['geometry'].area.apply(sq_ft_to_acres)

In [30]:
pb_poly.head()

,FIPS,COUSUB,TOWN_NAME,RPA,COUNTY,geometry,TOWN_ACRES
0,7215,84420,Whitefield,1,7,"POLYGON ((994238.463 696280.387, 994238.519 69...",22251.883271
1,7180,68980,Shelburne,1,7,"POLYGON ((1116173.456 706851.680, 1116176.409 ...",31212.735405
2,7075,18420,Dixville,1,7,"POLYGON ((1081680.490 901874.993, 1081432.971 ...",31455.275793
3,7070,18340,Dixs Grant,1,7,"POLYGON ((1098679.892 898611.462, 1098679.892 ...",12848.989387
4,7045,13780,Colebrook,1,7,"POLYGON ((1023456.031 890441.476, 1023456.031 ...",26108.861661


In [54]:
# Create counties DataFrame
counties = gpd.GeoDataFrame(pb_poly.groupby('COUNTY')['TOWN_ACRES'].sum())
tmp = []

for idx in counties.index:
    try:
        tmp.append((idx, pb_poly[pb_poly['COUNTY'] == idx]['geometry'].unary_union))
    except:
        print('Bad topo')
        
#     counties.loc[idx, 'geometry'] = pb_poly[pb_poly['COUNTY'] == idx]['geometry'].unary_union

TopologyException: Ring edge missing at 1040150.1627927045 272991.9652794578


1
3
5
7
9
11
13
Bad topo
15
17
19


In [48]:
counties

,TOWN_ACRES
COUNTY,
1,3.007972e+05
3,6.358022e+05
5,4.664760e+05
7,1.171982e+06
9,1.119728e+06
11,5.711344e+05
13,6.111431e+05
15,4.651785e+05
17,2.448425e+05


In [46]:
# counties = pb_poly.dissolve(by='COUNTY')

In [5]:
# m1 = geemap.Map(center=nh_center, zoom=nh_zoom)
# m1.add_basemap("HYBRID")
# hydro_url = "https://basemap.nationalmap.gov/arcgis/services/USGSHydroCached/MapServer/WMSServer?"
# m1.add_wms_layer(url=hydro_url,
#                  layers="0",  # Find all options in XML
#                  name="USGS Hydro",
#                  format="image/png",
#                  shown=True)
# m1

In [3]:
m = geemap.Map(center=nh_center, zoom=nh_zoom)

m.add_basemap("HYBRID")
m.add_basemap("NLCD 2016 CONUS Land Cover")
m

Map(center=[43.78699687528447, -71.51658744025995], controls=(WidgetControl(options=['position', 'transparent_…

## NH Trails

In [4]:
filepath = '/Users/heatherkusmierz/Documents/Files/Courses/Geography_Cartography/NH_Trails/GRANIT_20220201162014/'

points_path = os.path.join(filepath, 'nhtrails_points.shp')
lines_path = os.path.join(filepath, 'nhtrails.shp')

In [5]:
nh_trails_pts = gpd.read_file(points_path)
nh_trails = gpd.read_file(lines_path)

In [6]:
nh_trails_pts.head()

,OBJECTID,TYPE,POINAME,NOTES,geometry
0,1,Parking,None,None,POINT (820189.772 435001.368)
1,2,Shelter,None,None,POINT (925740.922 545624.824)
2,3,Shelter,Garfield Shelter,None,POINT (999551.772 616224.551)
3,4,Shelter,None,None,POINT (886406.780 483274.609)
4,5,Shelter,None,None,POINT (946212.081 558785.194)


In [7]:
nh_trails_pts.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2654 entries, 0 to 2653
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   OBJECTID  2654 non-null   int64   
 1   TYPE      2654 non-null   object  
 2   POINAME   337 non-null    object  
 3   NOTES     58 non-null     object  
 4   geometry  2654 non-null   geometry
dtypes: geometry(1), int64(1), object(3)
memory usage: 103.8+ KB


In [8]:
nh_trails_pts.crs

<Derived Projected CRS: PROJCS["NAD83(2011) / New Hampshire (ftUS)",GEOGCS ...>
Name: NAD83(2011) / New Hampshire (ftUS)
Axis Info [cartesian]:
- [east]: Easting (US survey foot)
- [north]: Northing (US survey foot)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Transverse Mercator
Datum: NAD83 (National Spatial Reference System 2011)
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [9]:
nh_trails.head()

,OBJECTID,TRAIL,TRAILNAME,TRAILSYS,COMMUNITY,MILES,BLAZE,MAINTORG,NOTES,PED,...,PAVED,GROOMED,ADA,WIDE,SEP_PATH,ALPINESKI,ACCURACY,MAPURL,SHAPE_Leng,geometry
0,1,Y,48T,Drummer Hill and Goose Pond,Keene,0.454302,None,0,None,1,...,None,None,None,None,None,None,1,https://www.trailfinder.info/trails/trail/drum...,2398.715561,"LINESTRING (821048.071 176910.411, 821012.921 ..."
1,2,Y,A-Z Trail,White Mountain National Forest,Bethlehem,3.331851,None,50110,None,1,...,None,None,None,None,None,None,2,https://www.fs.usda.gov/activity/whitemountain...,17592.174000,"LINESTRING (1030488.429 619271.548, 1030518.71..."
2,3,Y,Abanaki Quad,Attitash/Bear Peak,Bartlett,0.891730,None,0,None,1,...,None,None,None,1,None,1,3,None,4708.331831,"LINESTRING (1096078.672 576494.409, 1096075.00..."
3,4,Y,Abbott Brook Rd,None,Atkinson & Gilmanton,3.759935,None,0,None,None,...,None,None,None,1,None,None,2,None,19852.458011,"LINESTRING (1128608.984 904400.132, 1128604.01..."
4,5,Y,Abbott Brook Rd,None,Atkinson & Gilmanton,0.192031,None,0,None,1,...,None,None,None,1,None,None,2,None,1013.922527,"LINESTRING (1128610.233 903409.426, 1128607.76..."


In [10]:
nh_trails.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 19877 entries, 0 to 19876
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    19877 non-null  int64   
 1   TRAIL       19877 non-null  object  
 2   TRAILNAME   6849 non-null   object  
 3   TRAILSYS    10881 non-null  object  
 4   COMMUNITY   19877 non-null  object  
 5   MILES       19877 non-null  float64 
 6   BLAZE       274 non-null    object  
 7   MAINTORG    19877 non-null  int64   
 8   NOTES       194 non-null    object  
 9   PED         10071 non-null  object  
 10  MTNBIKE     1021 non-null   object  
 11  ROADBIKE    20 non-null     object  
 12  XCSKI       1056 non-null   object  
 13  SNOWMBL     3765 non-null   object  
 14  ATV         320 non-null    object  
 15  DIRTBIKE    0 non-null      object  
 16  HORSE       605 non-null    object  
 17  PADDLE      0 non-null      object  
 18  PAVED       0 non-null      object  
 

In [11]:
nh_trails.crs

<Derived Projected CRS: PROJCS["NAD83(2011) / New Hampshire (ftUS)",GEOGCS ...>
Name: NAD83(2011) / New Hampshire (ftUS)
Axis Info [cartesian]:
- [east]: Easting (US survey foot)
- [north]: Northing (US survey foot)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Transverse Mercator
Datum: NAD83 (National Spatial Reference System 2011)
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [12]:
pub_land_url = "https://opendata.arcgis.com/datasets/ac1d1c9b7fb548dcaaa6bdb5c80b70d5_6.geojson"


In [13]:
m2 = geemap.Map(center=nh_center, zoom=nh_zoom, ee_initialize=False)

m2.add_basemap("HYBRID")

m2

Map(center=[43.78699687528447, -71.51658744025995], controls=(WidgetControl(options=['position', 'transparent_…

In [15]:
style = {
    "stroke": True,
    "color": "#000000",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "#0000ff",
    "fillOpacity": 0.4,
}

In [16]:
m2.add_geojson(pub_land_url, style=style, layer_name="NH Public/Cnsvn Land")